In [6]:
import requests

api_key = "f8b1c3aa688e7f452ca16d3de75d9e81"
url = "https://api.openweathermap.org/data/2.5/weather"
params = {
    "q": "Prague,CZ",
    "appid": api_key
}

response = requests.get(url, params=params)
print(response.status_code)
print(response.text)

200
{"coord":{"lon":14.4208,"lat":50.088},"weather":[{"id":520,"main":"Rain","description":"light intensity shower rain","icon":"09n"}],"base":"stations","main":{"temp":283.59,"feels_like":283.14,"temp_min":282.75,"temp_max":284.19,"pressure":1021,"humidity":94,"sea_level":1021,"grnd_level":986},"visibility":10000,"wind":{"speed":5.14,"deg":230},"rain":{"1h":0.42},"clouds":{"all":100},"dt":1759779309,"sys":{"type":2,"id":2010430,"country":"CZ","sunrise":1759727394,"sunset":1759768242},"timezone":7200,"id":3067696,"name":"Prague","cod":200}


In [8]:
import requests

# ✅ Tvůj funkční OpenWeatherMap API klíč
API_KEY = "f8b1c3aa688e7f452ca16d3de75d9e81"

def get_weather(city: str, state: str | None = None, country: str | None = None) -> str:
    """
    Vrátí krátký popis počasí z OpenWeatherMap (např. 'Clouds', 'Rain', 'Clear').

    city:    Název města (povinné)
    state:   Kód státu/provincie (volitelné; používej jen pro USA)
    country: Kód země (např. 'CZ', 'US', 'GB') – volitelné
    """
    if not city:
        raise ValueError("Parametr 'city' je povinný!")

    # poskládání dotazu q podle dostupných informací
    parts = [city.strip()]
    if country:
        if state and country.upper() in {"US", "USA"}:
            parts.append(state.strip())
        parts.append(country.strip())

    q_value = ",".join(parts)

    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": q_value,
        "appid": API_KEY,
        "units": "metric",   # 🌡️ teploty v °C místo Kelvinů
        "lang": "en"         # nebo 'cz', pokud chceš české názvy
    }

    response = requests.get(url, params=params)
    data = response.json()

    if response.status_code != 200:
        raise Exception(f"API chyba: {data.get('message', 'Unknown error')} (HTTP {response.status_code})")

    return data["weather"][0]["main"]


In [9]:
# 1) jednorázově nastav proměnnou prostředí v aktuálním kernelu:
import os
os.environ["f8b1c3aa688e7f452ca16d3de75d9e81"] = "TVŮJ_API_KEY"

# 2) zavolej funkci (pár příkladů):
get_weather("Prague", country="CZ")        # např. 'Clouds'
get_weather("Portland", state="OR", country="US")
get_weather("Paris", country="FR")
get_weather("London")                      # když nezadáš zemi, může být méně jednoznačné


'Clouds'

In [10]:
get_weather("Prague", country="CZ")
# ➜ 'Clouds' (nebo jiný aktuální stav)

get_weather("London", country="GB")
get_weather("New York", state="NY", country="US")
get_weather("Tokyo", country="JP")


'Clouds'

In [11]:
import requests

API_KEY = "f8b1c3aa688e7f452ca16d3de75d9e81"

def get_weather_details(city: str, state: str | None = None, country: str | None = None) -> dict:
    """
    Vrátí slovník s počasím pro zadané město: 
    {'city': 'Prague', 'weather': 'Clouds', 'temp': 17.5, 'humidity': 63}

    Args:
        city:    Název města (povinné)
        state:   Kód státu/provincie (volitelné; jen pro USA)
        country: Kód země (např. 'CZ', 'US', 'GB') – volitelné
    """
    if not city:
        raise ValueError("Parametr 'city' je povinný!")

    # složení parametru q
    parts = [city.strip()]
    if country:
        if state and country.upper() in {"US", "USA"}:
            parts.append(state.strip())
        parts.append(country.strip())
    q_value = ",".join(parts)

    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": q_value,
        "appid": API_KEY,
        "units": "metric",
        "lang": "en"
    }

    response = requests.get(url, params=params)
    data = response.json()

    if response.status_code != 200:
        raise Exception(f"API chyba: {data.get('message', 'Unknown error')} (HTTP {response.status_code})")

    return {
        "city": city,
        "weather": data["weather"][0]["main"],
        "temp": round(data["main"]["temp"], 1),
        "humidity": data["main"]["humidity"]
    }


In [12]:
cities = [
    ("Oslo", "NO"),
    ("Prague", "CZ"),
    ("New York", "US"),
    ("Madrid", "ES"),
    ("London", "GB"),
    ("Paris", "FR")
]

results = [get_weather_details(city, country=country) for city, country in cities]

for r in results:
    print(f"{r['city']}: {r['weather']}, {r['temp']}°C, humidity {r['humidity']}%")


Oslo: Rain, 10.9°C, humidity 93%
Prague: Rain, 10.4°C, humidity 94%
New York: Clear, 25.6°C, humidity 59%
Madrid: Clear, 20.3°C, humidity 48%
London: Clouds, 16.8°C, humidity 77%
Paris: Clouds, 13.9°C, humidity 79%


In [13]:
# %pip install pandas requests -q

import requests
import pandas as pd

# ✅ Tvůj OpenWeather klíč
API_KEY = "f8b1c3aa688e7f452ca16d3de75d9e81"

def get_weather_details(city: str, state: str | None = None, country: str | None = None) -> dict:
    """
    Vrátí slovník s počasím pro zadané město:
    {'city': 'Prague', 'country': 'CZ', 'weather': 'Clouds', 'temp': 17.5, 'humidity': 63}
    """
    if not city:
        raise ValueError("Parametr 'city' je povinný!")

    parts = [city.strip()]
    if country:
        if state and country.upper() in {"US", "USA"}:
            parts.append(state.strip())
        parts.append(country.strip())
    q_value = ",".join(parts)

    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"q": q_value, "appid": API_KEY, "units": "metric"}
    resp = requests.get(url, params=params, timeout=10)

    try:
        data = resp.json()
    except ValueError:
        raise RuntimeError(f"Neočekávaná odpověď (HTTP {resp.status_code})")

    if resp.status_code != 200:
        msg = data.get("message", "Unknown error")
        raise RuntimeError(f"API chyba: {msg} (HTTP {resp.status_code})")

    return {
        "city": city,
        "country": (country or ""),
        "weather": data["weather"][0]["main"],
        "temp_c": round(data["main"]["temp"], 1),
        "humidity_pct": data["main"]["humidity"],
    }

def export_weather_to_csv(cities_with_country: list[tuple[str, str | None]], filepath: str = "weather_cities.csv") -> pd.DataFrame:
    """
    Pro seznam měst [(city, country), ...] stáhne počasí a uloží do CSV.
    Vrátí také DataFrame pro další práci v notebooku.
    """
    rows = []
    for city, country in cities_with_country:
        try:
            rows.append(get_weather_details(city, country=country))
        except Exception as e:
            # Když se něco nepovede (např. špatný název města), zapíšeme řádek s chybou
            rows.append({
                "city": city,
                "country": country or "",
                "weather": None,
                "temp_c": None,
                "humidity_pct": None,
                "error": str(e),
            })

    df = pd.DataFrame(rows, columns=["city", "country", "weather", "temp_c", "humidity_pct", "error"])
    # Uložíme s UTF-8 BOM, ať se to hezky otevře i v Excelu
    df.to_csv(filepath, index=False, encoding="utf-8-sig")
    return df


In [14]:
cities = [
    ("Oslo", "NO"),
    ("Prague", "CZ"),
    ("New York", "US"),
    ("Madrid", "ES"),
    ("London", "GB"),
    ("Paris", "FR"),
]

df = export_weather_to_csv(cities, filepath="weather_cities.csv")
df


,city,country,weather,temp_c,humidity_pct,error
0,Oslo,NO,Rain,10.3,93,NaN
1,Prague,CZ,Clouds,10.4,94,NaN
2,New York,US,Clear,25.7,59,NaN
3,Madrid,ES,Clear,20.0,49,NaN
4,London,GB,Clouds,16.8,77,NaN
5,Paris,FR,Clouds,13.9,80,NaN
